In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import *
from mycelia.shared.chain import * 
from mycelia.miner.train import * 
from mycelia.miner.model_io import *
from mycelia.shared.checkpoint import *
import bittensor as bt 

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subtensor = bt.Subtensor('test')
path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
miner_config = MinerConfig.from_path(path)

path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
vali_config = ValidatorConfig.from_path(path)

rank = 2
wallet, subtensor = setup_chain_worker(miner_config)
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(miner_config)
expert_manager = ExpertManager(miner_config)

2025-12-17 19:29:54 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-17 19:29:54 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


## Commit 

In [12]:
config = miner_config
# Read current version/hash snapshot
_, current_model_meta, _ = start_model_from(
    0,
    config,
    primary_ckpt_path=config.ckpt.validator_checkpoint_path,
    secondary_ckpt_path=config.ckpt.checkpoint_path,
)

current_model_meta.model_hash = 'xxx'

commits = get_chain_commits(config, subtensor)

max_model_meta = ModelMeta(global_ver=max(
    (
        c.global_ver
        for c, n in commits
        if c is not None and getattr(c, "global_ver", None) is not None
    ),
    default=0,
))

if current_model_meta is not None:
    logger.info(
        "Scan chain: Max model version on chain",
        max_model_version_on_chain=max_model_meta,
    )
    logger.info(
        "Scan chain: Local model version",
        current_model_version=current_model_meta,
    )
    max_model_meta = max(max_model_meta, current_model_meta)

    print('max_model_meta', max_model_meta)
    # 0) Download only from validator
    # commits = [(c, n) for c, n in commits if n.validator_permit]
    commits = [(c, n) for c, n in commits if getattr(c, "miner_seed", False)]
    commits = [(c, n) for c, n in commits if getattr(c, "model_hash", False)]

    # 1) collect candidates that are newer than the current version
    most_updated_commits = []  # type: ignore
    for (c, n) in commits:
        global_ver = getattr(c, "global_ver", 0)
        print(ModelMeta(global_ver = global_ver), max_model_meta)
        if ModelMeta(global_ver = global_ver) >= max_model_meta:
            most_updated_commits.append((c, n))


    if len(most_updated_commits) == 0:
        print("A", False, [])

    # 2) majority filter by model_hash among the newer candidates
    hash_counts = Counter([c.model_hash for c, n in most_updated_commits if getattr(c, "model_hash", False)])
    majority_hash, _count = hash_counts.most_common(1)[0]

    filtered_commits = [(c, n) for c, n in most_updated_commits if getattr(c, "model_hash", False) == majority_hash]
    if not filtered_commits:
        print("B", False, [])

    # 3) prepare download_meta for each entry (uid, ip, port, model_hash, model_version)
    download_meta = []
    for commit, neuron in filtered_commits:
        # Only include entries with reachable metadata
        try:
            download_meta.append(
                {
                    "uid": neuron.uid,
                    "ip": neuron.axon_info.ip,
                    "port": neuron.axon_info.port,
                    "model_hash": commit.model_hash,
                    "model_version": commit.model_version,
                    "target_hotkey_ss58": neuron.hotkey,
                }
            )

        except Exception:
            logger.info("Cannot append commit", commit=commit)

2025-12-17 19:37:51 [info     ] Get resume info from folder  [mycelia.shared.checkpoint] model_meta=ModelMeta(global_ver=120, inner_opt=0, path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid_16_hotkey_5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH_globalver_120'), role=None, model_hash=None) path={PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation')} result=found
2025-12-17 19:37:51 [info     ] Get resume info from folder  [mycelia.shared.checkpoint] model_meta=ModelMeta(global_ver=120, inner_opt=0, path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_120_inneropt_0'), role=None, model_hash=None) path={PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation')} result=found
2025-12-17 19:37:51 [info     ] Start model from ModelMeta(global_ver=120, inner_opt=0, path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/h

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [3]:
commits = get_chain_commits(miner_config, subtensor)
validator_seeds = get_validator_seed_from_commit(miner_config, commits)
# miners = get_miners_from_commit(config, commits)
# return assign_miners_to_validators(validator_seeds, miners)  #

for c in commits:
    print(c)

print(validator_seeds)

(ValidatorChainCommit(model_hash='b992b624a90e294c029e437b4a910538341ff68d40115309', global_ver=202, expert_group=0, miner_seed=28935, block=6056653), NeuronInfoLite(hotkey='5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH', coldkey='5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr', uid=16, netuid=348, active=False, stake=τ0.000000000, stake_dict={'5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr': τ0.000000000}, total_stake=τ0.000000000, rank=0.0, emission=0.0, incentive=0.0, consensus=0.0, trust=0.0, validator_trust=0.0, dividends=0.0, last_update=5750573, validator_permit=False, prometheus_info=PrometheusInfo(block=0, version=0, ip='0.0.0.0', port=0, ip_type=0), axon_info=AxonInfo( /ipv4/149.137.225.62:8001, 5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH, 5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr, 10000000 ), pruning_score=0, is_null=False))
(ValidatorChainCommit(model_hash='xxx', global_ver=None, expert_group=1, miner_seed=3024650, block=None), NeuronInfoLite(hotkey='5DUq

In [17]:
    max_model_meta = ModelMeta(global_ver=max(
        (
            c.global_ver
            for c, n in commits
            if c is not None and getattr(c, "global_ver", None) is not None
        ),
        default=0,
    ))

max_model_meta

ModelMeta(global_ver=202, inner_opt=0, path=None, role=None, model_hash=None)

In [15]:
max_model_version = max(
    (
        c.global_ver
        for c, n in commits
        if c is not None and getattr(c, "global_ver", None) is not None
    ),
    default=0,
)

max_model_meta = ModelMeta(global_ver=max_model_version)
max_model_meta
# print(max_model_version)

ModelMeta(global_ver=202, inner_opt=0, path=None, role=None, model_hash=None)

In [13]:
all_commitments = subtensor.get_all_commitments(netuid=miner_config.chain.netuid)

In [19]:
import json
ValidatorChainCommit.model_validate(json.loads(all_commitments['5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH']))

ValidatorChainCommit(model_hash=None, model_version=94, expert_group=0, miner_seed=62772, block=6047967)

In [22]:
for hotkey, commit in all_commitments.items():
    if hotkey != '5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH':
        continue

    print(hotkey)
    # try:
    status_dict = json.loads(commit)

    chain_commit = (
        ValidatorChainCommit.model_validate(status_dict)
        if "miner_seed" in status_dict
        else MinerChainCommit.model_validate(status_dict)
    )

5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH


ValidationError: 1 validation error for MinerChainCommit
block
  Field required [type=missing, input_value={'h': None, 'v': 94, 'e':...s': 62772, 'b': 6047967}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing